In [ ]:
import numpy as np
import csv
import zipfile
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math
from numpy import linalg as LA
from random import shuffle
import cvxpy as cp
import math
from collections import Counter
from statistics import mean
from sklearn.model_selection import GridSearchCV
import warnings
import random
import copy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso,LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import seaborn as sns
%matplotlib inline
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from numpy import genfromtxt
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from sklearn import linear_model
from sklearn import datasets

%matplotlib inline
plt.style.use('seaborn-white')

In [ ]:
# data generator

def iid_data(d, D, n, sigma):
    # matrix X generation
    XX = np.zeros((n, d + 1))
    for i in range(n):
        XX[i][0] = 1.0
        XX[i][1:] = np.random.normal(0, 1, d)

    # D/d sparse w generation
    w = [random.uniform(-1.0, 1.0) for i in range(D)]
    w = np.array(w)
    w = w / math.pow(LA.norm(w, 2), 2)
    main_dim = set([])
    while main_dim.__len__() < D:
        main_dim.add(random.randint(1, d - 1))
    main_dim = list(main_dim)
    main_dim.sort()
    ww = np.zeros((d + 1,))
    for i in range(D):
        ww[main_dim[i]] = w[i]

    # error generation
    error = np.random.normal(0, sigma, n)

    # Y generation
    YY = [np.matmul(np.transpose(ww), XX[i]) + error[i] for i in range(n)]
    YY = np.array(YY)

    return XX, YY, ww, main_dim,


def partition(XX, YY, N):
    X = []
    Y = []
    nn = int(XX.__len__() / N)
    for i in range(N):
        X.append(np.array(XX[i * nn:(i + 1) * nn]))
        Y.append(np.array(YY[i * nn:(i + 1) * nn]))

    return X, Y

In [ ]:
# evaluation

def MSE(XX, YY, w):
    return LA.norm(YY - np.matmul(XX, w), 2) / XX.__len__()

def FMeasure(w, main_dim):
    dim = []
    for i in range(w.__len__()):
        if w[i] != 0:
            dim.append(i)
    r = (set(dim) & set(main_dim)).__len__() / main_dim.__len__()
    p = (set(dim) & set(main_dim)).__len__() / (dim.__len__() + 0.000000001)

    return r,p, dim.__len__()/w.__len__()

def similarity(Ws, main_dim):
  dims = []
  MV = [0 for i in range(d+1)]
  for j in range(N):
    dims.append([])
    for i in range(d+1):
      if Ws[j][i] != 0:
        dims[j].append(i)
        MV[i] += 1
  a = [i for i in range(1001)]
  for i in range(N):
    a  = (set(a) & set(dims[i]))
  print('\n Intersection')
  print('intersect size: ' + str(a.__len__()))
  print((set(a) & set(main_dim)).__len__())


def MJ(ws):
  whole = []
  output = []
  for i in range(dims.__len__()):
    for j in range(dims[i].__len__()):
      whole.append(dims[i][j])

  for i in range(20):
      a = Counter(whole).most_common(1)[0][0]
      output.append(a)
      whole = list(filter((a).__ne__, whole))

  print((set(output) & set(main_dim)).__len__())

In [ ]:
# some functions need for the paper

def generating_M(X):
    d = X[0].__len__() - 1
    print('generating covariance matrix is on progress')  
    X = list(X)
    for i in range(X.__len__()):
        X[i] = list(X[i])
    for row in X:
            del row[0]
    gama = []
    # lambda_hat = [math.pow(10,i) * math.sqrt(math.log10(d)/( X.__len__())) for i in range(5)]
    lambda_hat = [math.pow(10, i-10) for i in range(15)]
    print('first for')
    for i in range(d):
        if i%10==0:
          print("*", end = '')
        x = [row[i] for row in X]
        XX = copy.deepcopy(X)
        for row in XX:
            del row[i]
        XX = np.array(XX)
        out = centralized(XX,x, lambda_hat, 10000)
        gama.append(out[0])

    C_hat = np.ones((d, d))
    print('\n senond for')
    for i in range(d):
        if i%10==0:
          print("*", end = '')      
        for j in range(d):
            if j < i:
              C_hat[i][j] = -1 * gama[i][j]
            elif i == j:
                C_hat[i][j] = 1
            else:
              C_hat[i][j] = -1 * gama[i][j-1]
    tai = []
    print('\n third for')
    for i in range(d):
        if i%10==0:
          print("*", end = '')
        x = [row[i] for row in X]
        XX = copy.deepcopy(X)
        for j in XX:
            del j[i]
        a = np.transpose(x - np.matmul(XX, gama[i]))
        a = np.matmul(a,x)
        tai.append(a / X.__len__())
    T_hat = np.diag(tai)
    temp = np.matmul(T_hat, C_hat)
    M = np.zeros((d+1,d+1))
    for i in range(d+1):
      for j in range(d+1):
        if i==0 or j == 0:
          if i == 0 and j == 0:
            M[i][j] = 1.0
        else:
          M[i][j] = temp[i-1][j-1]

    print('generating covariance matrix finished')  
    return M

In [ ]:
# optimizer
def centralized(X, Y, lambdas, itr):
    lasso = Lasso(alpha=0.1, max_iter=10000)
    parameters = {
        'alpha': lambdas}
    lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=10)
    lasso_regressor.fit(X, Y)
    lasso = Lasso(alpha=lasso_regressor.best_params_['alpha'], max_iter=itr)
    lasso.fit(X, Y)
    w = list(lasso.coef_)
    return w, lasso.alpha

def debiased(w, Y, X):
    w_d = Y - np.matmul(X, w)
    w_d = np.matmul(np.transpose(X), w_d)
    pre = generating_M(X)
    w_d = (np.matmul(pre, w_d)) / X.__len__()
    w_d = w + w_d
    return w_d

In [ ]:
# client update (Lasso)

lambdas = [math.pow(10, i-10) for i in range(15)]

def client_update(x, y, theta, lambdas):
    E = 5
    learning_rate = 0.005
    thetas = []
    mses = []
    for l in lambdas:
      for i in range(E):
        Yhat = x.dot(theta)
        delta = Yhat - y
        thetas.append(theta - learning_rate*(x.T.dot(delta) + l * np.sign(theta)))
        mses.append(MSE(x, y, thetas[thetas.__len__()-1]))
    best = thetas[mses.index(min(mses))]
    return best

In [ ]:
# retrieve data

X = genfromtxt('X.csv', delimiter=' ')
Y = genfromtxt('Y.csv', delimiter=' ')
main_w = genfromtxt('main_w.csv', delimiter=' ')
main_dim = genfromtxt('main_dim.csv', delimiter=' ')
wls = genfromtxt('wls.csv', delimiter=' ')

In [ ]:
# partitioning
N = 20
X_train, Y_train = partition(X, Y, N)

In [ ]:
# Federated Learning

lambdas = [math.pow(10, i-10) for i in range(15)]
C = 1
m = int(C * N)
w_f_l = np.ones((X[0].__len__(),))
t = 0
while True:
  print('Iteration: ' + str(t) + '\t', end='')
  S_t = []
  while S_t.__len__() <= m:
    S_t.append(random.randint(0,N))
    S_t = list(set(S_t))
  ws = []
  for i in range(m):
    k = S_t[i]
    ws.append(client_update(X_train[k], Y_train[k], w_f_l, lambdas))
  w_f_l = sum(ws)
  w_f_l = w_f_l/N
  MSE_f_l = MSE(X, Y, w_f_l)
  print(', Total MSE: ' + str(MSE_f_l))
  if MSE_f_l < 1.0338691186122174e-06 :
    break
  else:
    t += 1

f = [np.matmul(np.transpose(w_f_l), X[i]) for i in range(X.__len__())]
R2_f_l = r2_score(Y, f)
R_f_l, P_f_l, S_f_l = FMeasure(w_f_l, main_dim)

# save results
np.savetxt('w_f_l.csv', w_f_l, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)
f = open("Federated_Lasso_Results.txt", "a")
f.write('MSE: ' + str(MSE_f_l))
f.write('\nR2: ' + str(R2_f_l))
f.write('\nPrecision: ' + str(P_f_l))
f.write('\nRecall: ' + str(R_f_l))
f.write('\nSparsity: ' + str(S_f_l))
f.write('\nTotal number of iterations: ' + str(t))
f.close()

# Sparsity Checking
pp = []
ss = []
xx = []
for j in range(100):
  dim = []
  th = j * math.pow(10, -6.5)
  for i in range(w_f_l.__len__()):
          if math.fabs(w_f_l[i]) >= th:
              dim.append(i)
  p = (set(dim) & set(main_dim)).__len__() / (dim.__len__() + 0.000000001)
  s = dim.__len__()/w_f_l.__len__()
  pp.append(p)
  ss.append(s)
  xx.append(th)

plt.plot(xx, pp)
plt.plot(xx, ss)
plt.ylabel('Sparsity - Precision')
plt.xlabel('Threshold')
plt.show()